<a href="https://colab.research.google.com/github/GinuraAdikari/InsightHive/blob/Sentiment_Analysis/Model_ABSA_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer
import pandas as pd
import numpy as np

# Load preprocessed dataset
ABSA = pd.read_csv("ABSA_dataset.csv")

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
ABSA.head()

,tokens,aspect_mask,sentiment
0,"['one', 'best', 'game', 'music', 'soundtrack',...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",1
1,"['best', 'purchase', 'ever', 'bought', 'ex', '...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, ...",1
2,"['book', 'slow', 'weak', 'one', 'best', '##st'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, ...",0
3,"['must', '##rea', '##d', 'every', 'southern', ...","[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ...",1
4,"['horrible', 'watch', 'napoleon', 'want', 'fun...","[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0


In [ ]:
#------
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def convert_tokens_to_ids(row):
    """Converts tokenized words to BERT token IDs."""
    input_ids = tokenizer.convert_tokens_to_ids(eval(row["tokens"]))
    return input_ids

# Apply conversion
ABSA_data["input_ids"] = ABSA_data.apply(convert_tokens_to_ids, axis=1)

# Convert to PyTorch tensors for training
input_ids = torch.tensor(ABSA_data["input_ids"].tolist(), dtype=torch.long)
aspect_masks = torch.tensor(ABSA_data["aspect_mask"].tolist(), dtype=torch.long)
labels = torch.tensor(ABSA_data["sentiment"].tolist(), dtype=torch.long)

print("Tokens converted to input IDs for model training!")

In [ ]:
#------
from torch.nn.utils.rnn import pad_sequence

max_length = 128  # Set a fixed length for input sequences

# Apply padding
input_ids = pad_sequence([torch.tensor(x) for x in ABSA_data["input_ids"]], batch_first=True, padding_value=0)
aspect_masks = pad_sequence([torch.tensor(x) for x in ABSA_data["aspect_mask"]], batch_first=True, padding_value=0)
labels = torch.tensor(ABSA_data["sentiment"].tolist(), dtype=torch.long)

In [ ]:
from sklearn.model_selection import train_test_split

# Split into train (80%) and test (20%)
train_df, test_df = train_test_split(ABSA, test_size=0.2, random_state=42, stratify=ABSA["sentiment"])

# Save train and test datasets
train_df.to_csv("absa_train_dataset.csv", index=False)
test_df.to_csv("absa_test_dataset.csv", index=False)

print("Train and test datasets created!")

Train and test datasets created!


# Model input processing

In [ ]:
def convert_example_to_features(example, max_seq_length=128):
    tokens = eval(example["tokens"])
    aspect_mask = eval(example["aspect_mask"])

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Truncate if necessary
    if len(input_ids) > max_seq_length:
        input_ids = input_ids[:max_seq_length]
        aspect_mask = aspect_mask[:max_seq_length]

    # Pad shorter sequences
    pad_length = max_seq_length - len(input_ids)
    input_ids += [0] * pad_length  # PAD token
    aspect_mask += [0] * pad_length  # Pad aspect mask

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "input_mask": torch.tensor([1] * len(input_ids), dtype=torch.long),
        "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long)
    }

# Convert dataset into structured format
features = ABSA.apply(convert_example_to_features, axis=1)

# Extract tensors
input_ids = torch.stack([f["input_ids"] for f in features])
input_mask = torch.stack([f["input_mask"] for f in features])
aspect_mask = torch.stack([f["aspect_mask"] for f in features])
labels = torch.tensor(ABSA["sentiment"].values, dtype=torch.long)

# Create DataLoader for training
train_data = TensorDataset(input_ids, input_mask, aspect_mask, labels)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

# Create PyTorch DataLoader for evaluation
test_data = TensorDataset(input_ids, input_mask, aspect_mask, labels)
test_loader = DataLoader(test_data, batch_size=16)

print("Test dataset ready for evaluation!")

Test dataset ready for evaluation!


# Aspect-Aware BERT Model

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class BertForABSA(nn.Module):
    def __init__(self, bert_model=None, num_labels=2):
        super(BertForABSA, self).__init__()

        # Load BERT model (if not provided)
        self.bert = bert_model if bert_model else BertModel.from_pretrained("bert-base-uncased")

        # Aspect attention mechanism
        self.aspect_attention = nn.Linear(self.bert.config.hidden_size, 1)

        # Sentiment classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, aspect_mask, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        last_hidden_state = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)
        pooled_output = outputs.pooler_output  # Shape: (batch_size, hidden_size)

        # Compute attention scores for each token
        aspect_weights = torch.softmax(self.aspect_attention(last_hidden_state), dim=1)  # Shape: (batch_size, seq_len, 1)

        # Apply aspect mask at token level
        aspect_mask = aspect_mask.unsqueeze(-1).float()  # Expand to (batch_size, seq_len, 1)
        aspect_pooled = torch.sum(last_hidden_state * aspect_weights * aspect_mask, dim=1)  # Sum over sequence length

        logits = self.classifier(aspect_pooled)  # Final sentiment prediction
        return logits

# Handling Class Imbalance

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
labels_np = labels.cpu().numpy()
class_weights = compute_class_weight("balanced", classes=np.unique(labels_np), y=labels_np)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
import torch.optim as optim

# Load best hyperparameters
best_params = {"learning_rate": 2e-5, "batch_size": 16, "weight_decay": 0.01}

# Load balanced dataset
train_loader = DataLoader(balanced_train_data, batch_size=best_params["batch_size"], shuffle=True)

# Load model
model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=3)
optimizer = optim.AdamW(model.parameters(), lr=best_params["learning_rate"], weight_decay=best_params["weight_decay"])
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

# Training loop
for epoch in range(5):
    model.train()
    for batch in train_loader:
        input_ids, input_mask, aspect_mask, labels = batch

        optimizer.zero_grad()
        logits = model(input_ids, aspect_mask, input_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")


# Train the Model

In [ ]:
import torch.optim as optim

# Load model
bert_model = BertModel.from_pretrained("bert-base-uncased")
model = BertForABSA(bert_model=bert_model, num_labels=2)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

model.train()

# Training loop
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask, aspect_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, aspect_mask, attention_mask=attention_mask)
        logits = outputs  # assuming your model returns logits directly
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f"Epoch {epoch+1} Loss: {loss.item()}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1: Loss = 0.6686527132987976
Epoch 2: Loss = 0.4557132124900818
Epoch 3: Loss = 0.6118836998939514


In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer

# Load preprocessed dataset
df = pd.read_csv("absa_dataset.csv")

# Convert to tensors
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
input_ids = [tokenizer.convert_tokens_to_ids(eval(row["tokens"])) for _, row in df.iterrows()]
aspect_masks = [eval(row["aspect_mask"]) for _, row in df.iterrows()]
labels = df["sentiment"].values

input_ids = torch.tensor(input_ids, dtype=torch.long)
aspect_masks = torch.tensor(aspect_masks, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.long)

# Create DataLoader for training
train_data = TensorDataset(input_ids, aspect_masks, labels)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class BertForABSA(nn.Module):
    def __init__(self, num_labels=3):
        super(BertForABSA, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.aspect_attention = nn.Linear(self.bert.config.hidden_size, 1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, aspect_mask, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        pooled_output = outputs.pooler_output  # Correct way to access pooled output

        aspect_weights = torch.softmax(self.aspect_attention(pooled_output), dim=1)
        aspect_pooled = pooled_output * aspect_weights * aspect_mask.unsqueeze(-1)

        logits = self.classifier(aspect_pooled)
        return {"logits": logits}  # Wrap logits in a dictionary

In [ ]:
import torch
from transformers import BertTokenizer
from modeling_new import BertForABSA
import pandas as pd

# Load model & tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForABSA.from_pretrained("saved_model/absa_model.bin")
model.eval()

# Load test data
df = pd.read_csv("absa_dataset.csv")

input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(eval(row["tokens"])) for _, row in df.iterrows()], dtype=torch.long)
aspect_masks = torch.tensor([eval(row["aspect_mask"]) for _, row in df.iterrows()], dtype=torch.long)

with torch.no_grad():
    logits = model(input_ids, aspect_masks)
    predictions = torch.argmax(logits, dim=1)

df["Predicted_Sentiment"] = predictions.numpy()
df.to_csv("absa_predictions.csv", index=False)


# Evaluate the Model

In [ ]:
import torch
from sklearn.metrics import classification_report

# Load trained model
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Initialize BertForABSA with 2 sentiment labels (Binary Classification)
model = BertForABSA(bert_model=bert_model)
model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids, input_mask, aspect_mask, labels = batch
        outputs = model(input_ids, aspect_mask, input_mask)
        logits = outputs["logits"]  # Extract logits
        predictions = torch.argmax(logits, dim=1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print evaluation metrics
print("Model Performance on Test Set:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))

IndexError: too many indices for tensor of dimension 2

# Hyperparameter Optimization

In [ ]:
import torch
import torch.optim as optim
from sklearn.model_selection import ParameterGrid

# Define possible hyperparameters
param_grid = {
    "learning_rate": [2e-5, 3e-5, 5e-5],
    "batch_size": [8, 16],
    "weight_decay": [0.01, 0.001]
}

# Create all possible combinations
param_combinations = list(ParameterGrid(param_grid))

best_model = None
best_val_loss = float("inf")

for params in param_combinations:
    print(f"\nTraining with params: {params}")

    # Load model
    model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=3)
    optimizer = optim.AdamW(model.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
    criterion = torch.nn.CrossEntropyLoss()

    train_loader = DataLoader(train_data, batch_size=params["batch_size"], shuffle=True)
    val_loader = DataLoader(val_data, batch_size=params["batch_size"])

    for epoch in range(3):
        model.train()
        for batch in train_loader:
            input_ids, input_mask, aspect_mask, labels = batch
            optimizer.zero_grad()
            logits = model(input_ids, aspect_mask, input_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

    # Validate the model
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, aspect_mask, labels = batch
            logits = model(input_ids, aspect_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss}")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = model

# Save the best model
best_model.save_pretrained("best_absa_model")
print("Best model saved!")


# Model Evaluation: Precision, Recall, F1-Score

In [ ]:
max_length = max(len(eval(row["tokens"])) for _, row in df_test.iterrows())

print(f"Automatically set max_length to: {max_length}")

NameError: name 'df_test' is not defined

In [ ]:
import torch
import pandas as pd
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Initialize BertForABSA with Binary Classification (2 sentiment labels)
model = BertForABSA(bert_model=bert_model, num_labels=2)
model.eval()

# Load test data
df_test = pd.read_csv("absa_test_dataset.csv")

# Set maximum sequence length (cannot exceed 512 for BERT)
MAX_LENGTH = min(512, max(len(eval(row["tokens"])) for _, row in df_test.iterrows()))
print(f"Truncating all sequences to max_length={MAX_LENGTH}")

# Convert test dataset to tensors (Token IDs & Aspect Masks)
input_ids_list = [tokenizer.convert_tokens_to_ids(eval(row["tokens"]))[:MAX_LENGTH] for _, row in df_test.iterrows()]
aspect_masks_list = [eval(row["aspect_mask"])[:MAX_LENGTH] for _, row in df_test.iterrows()]

# Apply padding to ensure uniform size
input_ids = pad_sequence([torch.tensor(seq) for seq in input_ids_list], batch_first=True, padding_value=0)
aspect_masks = pad_sequence([torch.tensor(mask) for mask in aspect_masks_list], batch_first=True, padding_value=0)

# Create Attention Mask (1 for real tokens, 0 for padding)
attention_masks = (input_ids != 0).long()

# Convert sentiment labels to tensor
labels = torch.tensor(df_test["sentiment"].values, dtype=torch.long)

# Create DataLoader
test_data = TensorDataset(input_ids, attention_masks, aspect_masks, labels)
test_loader = DataLoader(test_data, batch_size=16)

# Run evaluation
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, aspect_mask, labels = batch

        # **Pass attention_mask to model**
        outputs = model(input_ids, aspect_mask, attention_mask=attention_mask)  # ✅ Returns a tensor, not a dict
        logits = outputs  # ✅ Correct: use `outputs` directly

        predictions = torch.argmax(logits, dim=1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print classification report
print("Model Performance:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))


Truncating all sequences to max_length=512
Model Performance:
              precision    recall  f1-score   support

    Negative       0.30      0.85      0.45        82
    Positive       0.62      0.11      0.19       180

    accuracy                           0.34       262
   macro avg       0.46      0.48      0.32       262
weighted avg       0.52      0.34      0.27       262

